In [1]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from fbprophet import Prophet
import json 
import os
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from timeit import default_timer as timer
import warnings
warnings.filterwarnings("ignore")
import pickle
pkl_path = r"..\Results\FbProphet\Prophet.pkl"

def timeseries_evaluation_metrics_func(y_true, y_pred):
    
    def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('Evaluation metric results:-')
    mse = metrics.mean_squared_error(y_true, y_pred)
    mae = metrics.mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(metrics.mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)
    print(f'MSE is : {mse}')
    print(f'MAE is : {mae}')
    print(f'RMSE is : {rmse}')
    print(f'MAPE is : {mape}')
    print(f'R2 is : {r2}',end='\n\n')
    return {'mse' : mse, 'mae' : mae, 'rmse' : rmse, 'mape' : mape, 'r2' : r2}

df = pd.read_csv(r'..\Data\Panama Electricity\continuous_dataset_preprocessing.csv', parse_dates= True)
# df["datetime"] = df["datetime"].apply(pd.to_datetime)
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.rename(columns={'datetime': 'ds', 'nat_demand': 'y'})


column_names = df.columns.tolist()
train_column_names = df.columns.tolist()
column_names.remove('ds')
train_column_names.remove('ds')
train_column_names.remove('y')

In [2]:
# valid = df[df['datetime'] <= '1/1/2019 23:00']
# valid = valid[valid['datetime'] > '1/1/2018 23:00']
train = df[df['ds'] <= '1/1/2019 23:00']
test = df[df['ds'] > '1/1/2019 23:00']

In [13]:
m = Prophet( yearly_seasonality=True)
for i in train_column_names:
    m.add_regressor(i)

In [15]:
m.fit(train)

D:\Programming\Anaconda\envs\DL_project\lib\site-packages\fbprophet\forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


In [18]:
import pickle
pkl_path = r"..\Results\FbProphet\Prophet.pkl"

with open(pkl_path, "wb") as f:
    # Pickle the 'Prophet' model using the highest protocol available.
    pickle.dump(m, f)

In [3]:
import pickle
pkl_path = r"..\Results\FbProphet\Prophet.pkl"
with open(pkl_path, 'rb') as f:
    m = pickle.load(f)

In [4]:
p = 100
future = m.make_future_dataframe(periods=p,freq='H',include_history=False)

In [5]:
var_predictions = pd.read_csv(r'../Results/VAR/res_100.csv', parse_dates= True)

In [6]:
for i in train_column_names:
    if (i == 'holiday' or i == 'school'):
        future[i] = test[i].values[:100] 
        continue
    future[i] = var_predictions[i].values[:100] 

In [7]:
forecast = m.predict(future)

In [8]:
folder_save = f'..\Results\FbProphet'
for horizon in [6, 12, 24, 36, 48, 60]:
    print(f'Evaluation metric for {i} in horizon {horizon}')
    metrics_test = timeseries_evaluation_metrics_func(test['y'][:horizon],forecast['yhat'].values[:horizon])
    with open(f"{folder_save}\metrics_test_{horizon}.json", "w") as outfile:
        json.dump(metrics_test, outfile)
    print()

Evaluation metric for school in horizon 6
Evaluation metric results:-
MSE is : 362820.42161985155
MAE is : 602.1724885582368
RMSE is : 602.3457658354141
MAPE is : 62.50356584523633
R2 is : -537.1177701761627


Evaluation metric for school in horizon 12
Evaluation metric results:-
MSE is : 398084.0503729456
MAE is : 629.5084149113817
RMSE is : 630.9390227057966
MAPE is : 57.73180344733964
R2 is : -9.456841722771243


Evaluation metric for school in horizon 24
Evaluation metric results:-
MSE is : 454943.69374093675
MAE is : 671.8291597654005
RMSE is : 674.4951398942299
MAPE is : 55.222682118718645
R2 is : -10.074270575717664


Evaluation metric for school in horizon 36
Evaluation metric results:-
MSE is : 447260.67563024006
MAE is : 666.5551670270519
RMSE is : 668.7755046577589
MAPE is : 56.068887343631225
R2 is : -10.048361318232498


Evaluation metric for school in horizon 48
Evaluation metric results:-
MSE is : 464958.14613733435
MAE is : 679.3318850389127
RMSE is : 681.8783954176392


In [27]:
timeseries_evaluation_metrics_func(test['y'][:100],forecast['yhat'].values[:100])

Evaluation metric results:-
MSE is : 3146.399889968986
MAE is : 41.64173921601396
RMSE is : 56.092779303302365
MAPE is : 3.631844101207703
R2 is : 0.905805418571187



{'mse': 3146.399889968986,
 'mae': 41.64173921601396,
 'rmse': 56.092779303302365,
 'mape': 3.631844101207703,
 'r2': 0.905805418571187}

In [41]:
train

,ds,y,T2M_toc,QV2M_toc,TQL_toc,W2M_toc,T2M_san,QV2M_san,TQL_san,W2M_san,T2M_dav,QV2M_dav,TQL_dav,W2M_dav,holiday,school
0,2015-01-03 01:00:00,970.3450,25.865259,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.096100,5.364148,0,0
1,2015-01-03 02:00:00,912.1755,25.899255,0.018653,0.016418,22.166944,23.399255,0.017265,0.001327,10.681517,22.578943,0.016509,0.087646,5.572471,0,0
2,2015-01-03 03:00:00,900.2688,25.937280,0.018768,0.015480,22.454911,23.343530,0.017211,0.001428,10.874924,22.531030,0.016479,0.078735,5.871184,0,0
3,2015-01-03 04:00:00,889.9538,25.957544,0.018890,0.016273,22.110481,23.238794,0.017128,0.002599,10.518620,22.512231,0.016487,0.068390,5.883621,0,0
4,2015-01-03 05:00:00,893.6865,25.973840,0.018981,0.017281,21.186089,23.075403,0.017059,0.001729,9.733589,22.481653,0.016456,0.064362,5.611724,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35034,2019-01-01 19:00:00,1126.2821,25.780603,0.017469,0.011211,24.981605,25.124353,0.017592,0.005699,10.515848,23.436853,0.016508,0.070129,5.756809,1,0
35035,2019-01-01 20:00:00,1148.1311,25.555078,0.017385,0.009068,24.418105,24.703516,0.017492,0.004416,10.727559,23.047266,0.016500,0.072906,6.004097,1,0
35036,2019-01-01 21:00:00,1140.0793,25.376306,0.017379,0.010250,23.967645,24.399744,0.017333,0.004755,11.202762,22.837244,0.016471,0.071960,6.156340,1,0
35037,2019-01-01 22:00:00,1107.0909,25.226678,0.017393,0.012093,23.037614,24.101678,0.017088,0.002463,11.633762,22.695428,0.016371,0.065735,6.386354,1,0


In [19]:
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from timeit import default_timer as timer
import warnings
warnings.filterwarnings("ignore")